# LAB 2: Feature Representation

The goals of this lab are the following :
- Making data linearly separable by utilizing polynomial features
- Understanding the relation between model complexity and performance, underfitting, overfitting
- Understanding how to deal with Categorical variables and preprocess features

This lab is greatly inspired by those notes on feature representation, feel free to check it out : https://openlearninglibrary.mit.edu/assets/courseware/v1/b5ca509c17bab346cc6252ca41a1aac7/asset-v1:MITx+6.036+1T2019+type@asset+block/notes_chapter_Feature_representation.pdf

## Imports and helper functions

In [ ]:
import numpy as np
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_blobs, make_moons, make_circles

In [ ]:
def plot_2D_data(X, y, ax, axlims =None):
    '''
    Plot a 2D dataset
    
    Parameters :
    ------------
     X: np.ndarray, shape (num_samples, num_features)
            An input data matrix, where each row is an input vector and each
            column is a feature. num_features should be equal to two
        
     y: np.ndarray, shape (num_samples,)
            The class label vector for the training data. Each element
            corresponds to the class label for the corresponding row in X. All the labels should be 1 or -1
    '''
    num_samples, num_features = X.shape
    assert num_features == 2, f'X should be of shape (num_samples, 2) but is actually of shape {X.shape}'
    x1 = X[:, 0]
    x2 = X[:, 1]
    
    
    ax.scatter(x1[y==1], x2[y==1],c='b', edgecolors='k', linewidth=.5, label="1")
    ax.scatter(x1[y==-1], x2[y==-1],c='r', marker='x' ,edgecolors='k', linewidth=.5, label="-1")
    
    if axlims is None:
        xlim, ylim = ax.get_xlim(), ax.get_ylim()
        xmin = xlim[0] -0.1 *(xlim[1] - xlim[0])
        xmax = xlim[1] +0.1 *(xlim[1] - xlim[0])
        ymin = ylim[0] -0.1 *(ylim[1] - ylim[0])
        ymax = ylim[1] +0.1 *(ylim[1] - ylim[0])
    else:
        xmin, xmax, ymin, ymax = axlims
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    
    
    ax.set_title('Dataset')
    ax.set_xlabel('$x_1$')
    ax.set_ylabel('$x_2$')
    ax.legend(loc='best')
    return ax
    
def plot_2D_linear_decision_boundaries(X, y, coef, bias, ax, axlims=None):
    ax = plot_2D_data(X, y, ax, axlims=axlims)
    xlim, ylim = ax.get_xlim(), ax.get_ylim()
    xx = np.linspace(xlim[0],xlim[1], 10)
    if coef[1] != 0:
        yy = - (coef[0] * xx + bias) / coef[1]
        ax.plot(xx, yy, c='purple', label = 'Decision boundary')
    elif coef[0] != 0:
        plt.axvline(x = - coef[0] / bias, color='purple', label=f'Decision boundary')
    ax.legend(loc='best')
    return ax

def plot_2D_polynomial_decision_boundaries(X, y, get_polynomial_features, k, coef, ax, axlims=None):
    ax = plot_2D_data(X, y, ax, axlims=axlims)
    xlim, ylim = ax.get_xlim(), ax.get_ylim()
    u = np.linspace(xlim[0], xlim[1], 100)
    v = np.linspace(ylim[0], ylim[1], 100)
    U, V = np.meshgrid(u, v)
    
    
    U = np.ravel(U)
    V = np.ravel(V)
    
    X_poly = get_polynomial_features(np.stack([U,V], axis=1),k=k)
    Z = np.dot(X_poly, coef)
    U = U.reshape((len(u),len(v)))
    V = V.reshape((len(u),len(v)))
    Z = Z.reshape((len(u),len(v)))
    db = ax.contour(U,V,Z, levels=[0], colors='purple')
    db.collections[0].set_label('Decision Boundary')
    ax.legend(loc='best')
    clf_regions = ax.contourf(U,V,Z, levels=[Z.min()-1,0,Z.max()+1], colors=['r','b'], alpha=0.1)
    return ax
    

# Polynomial features

We have seen in the previous Practical session that when the data is linearly separable, the perceptron algorithm converges and finds a linear separation of the data. However, when the data is not linearly separable, this algorithm might not be able to produce a meaningful result. Let's look at some examples here :

## A linearly separable dataset

In [ ]:
X, y = make_blobs(centers=2, random_state=10)
y[y==0] =-1

In [ ]:
_, ax = plt.subplots()
ax = plot_2D_data(X,y,ax)

In [ ]:
clf = Perceptron()
clf.fit(X,y)
coef, intercept = clf.coef_[0], clf.intercept_
print(f'Coefficients : {coef}')
print(f'bias : {intercept}')

In [ ]:
_, ax = plt.subplots()
ax = plot_2D_linear_decision_boundaries(X, y, coef, intercept, ax)

## The XOR dataset

In [ ]:
X = np.array([[i,j] for i in[-1,1] for j in [-1,1]])
y = X[:,0] * X[:,1]
# Add some noise
X = 1.0 * X +  0.01 * np.random.rand(4,2) 
print(f'The data points are :\n {X}')
print(f'Their corresponding labels are {y}')

In [ ]:
_, ax = plt.subplots()
ax = plot_2D_data(X, y, ax)

Q: Is this dataset linearly separable ?

In [ ]:
clf = Perceptron()
clf.fit(X,y)
coef, intercept = clf.coef_[0], clf.intercept_
print(f'Coefficients : {coef}')
print(f'bias : {intercept}')

In [ ]:
_, ax = plt.subplots()
ax = plot_2D_linear_decision_boundaries(X, y, coef, intercept, ax)

## A 1D dataset 

We consider a very simple 1D dataset with 4 datapoints. 

In [ ]:
X = np.array([-3, -1, 1.25, 2])
# We add a second coordinate equal to 0 for easier visualization
X_aug_0 = np.stack([X, np.zeros_like(X)], axis=1)
y = np.array([1, -1, -1, 1])

_, ax = plt.subplots()
ax = plot_2D_data(X_aug_0, y, ax)
ax.set_ylabel('$x_2 = 0$')

Q: What is a linear boundary for a 1D dataset ?

Q: Is this dataset Linearly separable ?

### Naïve Approach

In [ ]:
clf = Perceptron()
clf.fit(X_aug_0,y)
coef0, intercept0 = clf.coef_[0], clf.intercept_
print(f'Coefficients : {coef0}')
print(f'bias : {intercept0}')

In [ ]:
_, ax = plt.subplots()
ax = plot_2D_linear_decision_boundaries(X_aug_0, y, coef0, intercept0, ax)

### A smarter way to proceed 

What if we could put the data in a 2D space in a way that would make the data linearly separable ? One way is to consider the transformation $\Phi(x) = (x, x^2)$

In [ ]:
X_aug_poly = np.stack([X, X**2], axis= 1)
_, ax = plt.subplots()
ax = plot_2D_data(X_aug_poly, y, ax)
ax.set_ylabel('$x^2$')
pass

Q : Is this new dataset lineary separable ?

In [ ]:
# TODO Train a perceptron on the augmented dataset
# coef_poly =
# intercept_poly = 
# print(f'Coefficients : {coef_poly}')
# print(f'bias : {intercept_poly}')

In [ ]:
# plot the resulting boundary
_, axes = plt.subplots(1,2, figsize=(10,5))
plt.subplots_adjust(right=1.5, top=1.5)
#Plot new separation
axes[0] = plot_2D_linear_decision_boundaries(X_aug_poly, y, coef_poly, intercept_poly, axes[0])
axes[0].set_xlabel('x')
axes[0].set_ylabel('$x^2$')
axes[0].set_title('Polynomial Features')

#Plot old separation
axes[1] = plot_2D_linear_decision_boundaries(X_aug_0, y, coef0, intercept0, axes[1])
axes[1].set_xlabel('x')
axes[1].set_ylabel('$x_2 = 0$')
axes[1].set_title('Basic Features')

plt.show()

*Tips :* plt.subplots(n_rows, n_columns) allows you to easily create multiple plots on the same figure !

Q : Represent the decision Boundary on the 1D Dataset.

## Polynomial Features


If your dataset has $D$ numerical features, a systematic strategy is to use a polynomial basis. The polynomial basis of order $k$ (where $k$ is a positive integer) contains all products of features of degree inferior or equal to $k$.

For example the monome $x_1x_2^3$ is of degree $4$ and the monome $x_1x_2x_3^4$ is of degree 6

The following table illustrates the $k$-th order polynomial basis for different values of $k$:

\begin{array}{|c|c|c|}
\hline \\
\text{Order} k & D=1 & D>1 \\
\hline \\
0 &[1] & [1]\\
1 & [1,x]&[1,x_1,\ldots,x_d] \\
2 & [1,x,x^2] & [1,x_1,\ldots,x_d, x_1^2,x_1x_2,\ldots]\\
3 & [1,x,x^2,x^3]& [1,x_1,\ldots,x_d, x_1^2,x_1x_2,\ldots, x_1^3,x_1^2x_2,x_1,x_2,x_3,\ldots]\\ 
\vdots & \vdots & \vdots \\
\end{array}


For example, if you consider a polynomial basis or order 2 for D=2 features, the corresponding polynomial features are $[1,x_1,x_2,x_1^2,x_1x_2,x_2^2]$

Bonus Question : If you have 2 features (D=2), how many features are there in the $k$-th order polynomial basis? Same question for a general $D$.

Here, you should code the `get_polynomial_features` function that will create the polynomial basis of order $k$ for a 2D dataset

In [ ]:
# Generate a random dataset of size(10,2)
X = np.random.rand(10,2)

def get_polynomial_features(X, k=2):
    """
    Creates the polynomial features for a 2D dataset X.
    
    Parameters:
    -----------
    X: np.ndarray of shape (num_samples, 2)
    An input data matrix where each row is an input vector and each column is a feature.
    
    k : int
    The order of the polynomial basis
    
    Returns:
    --------
    X_aug: np.ndarray of shape (num_samples, dimension of polynomial basis of order k)
    An augmented data matrix containing all the polynomial features of the polynomial basis of order k
    """
    
    ### TO CODE
    return X_aug

X_aug = get_polynomial_features(X, k=2)
print(f'The original dataset was of shape {X.shape}')
print(f'The augmented dataset is of shape {X_aug.shape}')
assert X_aug.shape[1] == 6, f'X_aug is of shape {X_aug.shape} but there should be 6 features for a polynomial basis of order 2 with 2 original features'
    

## Back to the XOR dataset

In [ ]:
X = np.array([[i,j] for i in[-1,1] for j in [-1,1]])
y = X[:,0] * X[:,1]
# Add some noise
X = 1.0 * X +  0.01 * np.random.rand(4,2) 
print(f'The data points are :\n {X}')
print(f'Their corresponding labels are {y}')

In [ ]:
_, ax = plt.subplots()
ax = plot_2D_data(X, y, ax)

In [ ]:
clf = Perceptron()
clf.fit(X,y)
coef0, intercept0 = clf.coef_[0], clf.intercept_
print(f'Coefficients : {coef}')
print(f'bias : {intercept}')

In [ ]:
_, ax = plt.subplots()
ax = plot_2D_linear_decision_boundaries(X, y, coef, intercept, ax)

We are now going to make this dataset linearly separable by transforming it into a higher dimensional dataset using our polynomial features. We can then use the perceptron algorithm on the augmented dataset to solve the classification problem. Let's try it with a polynomial basis of order $k=2$. In that case the feature transformation is $$\phi([x_1,x_2]) =[1,x_1,x_2,x_1^2,x_1x_2,x_2^2]$$

Q : With that feature transformation, do we need a bias in the perceptron algorithm ?

In [ ]:
# TODO : Add polynomials feature to the dataset
# X_aug =

# TODO : Train a perceptron on the augmented dataset
# clf = Perceptron(fit_intercept=False)
# coef_XOR = 

# Plot the resulting decision boundary
_, axes = plt.subplots(1,2, figsize = (10,5))

#Plot Separation with polynomial Features
axes[0] = plot_2D_polynomial_decision_boundaries(X, y, get_polynomial_features, k=2, coef=coef_XOR, ax= axes[0])
axes[0].set_title('Polynomial Features')

#Plot Separation without Polynomial Features
axes[1] = plot_2D_linear_decision_boundaries(X, y, coef0, intercept0, axes[1])
axes[1].set_xlabel('x')
axes[1].set_title('Basic Features')

plt.show()

## The Moon Dataset

We now move our focus to another non linearly separable dataset.

In [ ]:
X,y = make_moons(n_samples = 400, noise =0.1, random_state=42,)
y[y!=1] = -1
_, ax = plt.subplots()

ax = plot_2D_data(X, y, ax)

In [ ]:
# TODO Compute the polynomial features of order k, fit a perceptron, compute the number of errors and plot the associated decision boundary for k taking values in [1,2,3,4,5,6]. 
_, axes = plt.subplots(2,3, figsize = (15,10))
plt. subplots_adjust(left=None, bottom=None, right=1.5, top=1.5, wspace=None, hspace=None)
list_degrees = [1,2,3,4,5,6]
for k, ax in zip(list_degrees, axes.flatten()):
    # TODO Compute the polynomial features of order k
    
    # TODO Fit a Perceptron to the polynomial features and get the coefficients
    # coef_moon =
    
    # TODO Compute the number of errors
    # n_errors = 
    ax = plot_2D_polynomial_decision_boundaries(X, y, get_polynomial_features, k=k, coef=coef_moon, ax= ax)
    ax.set_title(f'Polynomial Features of order {k}\n Number of errors : {n_errors}')

Q : Which value of $k$ should we choose here? Why ?

## An Introduction to Generalization error 

In Machine Learning, usually the objective is not only to fit existing data with our model but also to make good predictions on new data that the model has never seen before. We take a look at the following blob datasets :

In [ ]:
X, y = make_blobs(n_samples = 120, centers=2, cluster_std=5, random_state=48)
y[y!=1] = -1
_, ax = plt.subplots()

ax = plot_2D_data(X, y, ax)

Here we will use another model we will learn about in future lectures, logistic regression, that performs better than the Perceptron algorithm on non Linearly separable data.

In [ ]:
# TODO Compute the polynomial features of order k, fit a Logistic Regression, compute the number of errors and plot the associated decision boundary for k taking values in [1,2,3,4,5,6]. 
_, axes = plt.subplots(2,3, figsize = (15,10))
plt. subplots_adjust(left=None, bottom=None, right=1.5, top=1.5, wspace=None, hspace=None)
list_degrees = [1,2,3,4,5,6]
for k, ax in zip(list_degrees, axes.flatten()):
    # TODO Compute the polynomial features of order k
    
    # TODO Fit a Logistic Regression to the polynomial features and get the coefficients
    # clf = LogisticRegression(fit_intercept=False, max_iter=10000, random_state=42)
    # coef_blob =
    
    # TODO Compute the number of errors
    # n_errors = 
    ax = plot_2D_polynomial_decision_boundaries(X, y, get_polynomial_features, k=k, coef=coef_blob, ax= ax)
    ax.set_title(f'Polynomial Features of order {k}\n Number of errors : {n_errors}')

Q : Keeping in mind that we want our model to generalize well (meaning making relevant predictions on data points that we have not seen yet but that follow the same distribution that the data points we have seen), what do you think is the best value of $k$ to pick here?

A very useful tool to measure the generalization of a model is to use a training set and a validation set. We train the model on the training set and then evaluate its performance on the validation set (with new data different than the one used during training). Here we split the data into Training set and Validation set.

In [ ]:
# Get the min and max values of the data :
xmin = X[:,0].min() - 0.1*(X[:,0].max()-X[:,0].min())
xmax = X[:,0].max() + 0.1*(X[:,0].max()-X[:,0].min())
ymin = X[:,1].min() - 0.1*(X[:,1].max()-X[:,1].min())
ymax = X[:,1].max() + 0.1*(X[:,1].max()-X[:,1].min())
axlims=(xmin, xmax, ymin, ymax)
# Split the data into two groups, training and validation
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.33, random_state=42)

_, axes = plt.subplots(1,3, figsize = (15,5))
for ax, X_sub, y_sub, name in zip(axes, [X, X_train, X_val], [y, y_train,y_val], ['Full set', 'Training set', 'Validation set']):
    ax = plot_2D_data(X_sub, y_sub, ax,axlims=axlims)
    ax.set_title(name)

In [ ]:
# TODO Compute the polynomial features of order k, fit a perceptron on the training set and plot the associated decision boundary for k taking values in [1,2,3,4,5,6] for both the training and validation sets.
list_degrees = [1,2,3,4,5,6]
_, axes = plt.subplots(len(list_degrees),2, figsize = (15,15))
plt.subplots_adjust(left=None, bottom=None, right=1.5, top=1.5, wspace=None, hspace=None)
for i,k in enumerate(list_degrees):
    # TODO Compute the polynomial features of order k for both Training and validation sets
    
    # TODO On the training set, fit a Logistic Regression to the polynomial features and get the coefficients
    # coef =
    
    # TODO Compute the number of errors for both training and validation
    # n_errors_train =
    # n_errors_valid =
    axes[i,0] = plot_2D_polynomial_decision_boundaries(X_train, y_train, get_polynomial_features, k=k, coef=coef, ax= axes[i,0],axlims=axlims)
    axes[i,0].set_title(f'Training set: Polynomial Features of order {k}\n Number of training errors : {n_errors_train}')
    axes[i,1] = plot_2D_polynomial_decision_boundaries(X_val, y_val, get_polynomial_features, k=k, coef=coef, ax= axes[i,1],axlims=axlims)
    axes[i,1].set_title(f'Validation set: Polynomial Features of order {k}\n Number of validation errors : {n_errors_valid}') 

Q : Using this new plot, which is the best value of $k$? Why ?

# Categorical Variables and Feature preprocessing


Here we will take a look at the dataset from the **Titanic - Machine Learning from Disaster** Dataset from the following Kaggle competition: https://www.kaggle.com/competitions/titanic/data  The goal of this competition is to use machine learning to predict the survival or death of passengers using available features(such as gender, age, ...). We strongly recommend that you try making a submission to this competition as an exercise, it's a really nice way to get your hands on real data and see more of the machine learning pipeline. We recommend this notebook to get started (https://www.kaggle.com/code/alexisbcook/titanic-tutorial/notebook). You might want to come back to the competition from time to time when you see new methods in class to see how they improve your results!

You should upload to the google colab (or your local machine) the files `gender_submission.csv`, `train.csv`, test.csv that you can find on Aula Global.

To deal with `.csv` files, a very useful `Python` library is `Pandas`:

In [ ]:
import pandas as pd

We start by reading the different files, we can use the read_csv method to obtain a Dataframe a python object containing the content of the csv file. We can use the **head** method to see the first rows of the data.

In [ ]:
train = pd.read_csv('train.csv')
train.head()

In [ ]:
test = pd.read_csv('test.csv')
test.head()

As you can see, the `test.csv` file contains the information of passengers for which you want to make a prediction. The Survived column is not present is this `dataFrame`, indeed the goal of our model is to make prediction on data for which it doesn't know the target values, data that we can't use to train the model.

In [ ]:
gender_submission = pd.read_csv('gender_submission.csv')
gender_submission.head()

The gender_submission `dataFrame` contains a prediction on the test set that predicts that every woman survived and every men died. We can look at the training data to see the survival rate of men and women on the titanic :

In [ ]:
print(f"Rate of survival of men : {train[train.Sex == 'male'].Survived.mean()}")
print(f"Rate of survival of women : {train[train.Sex == 'female'].Survived.mean()}")

As we can see, this is not a bad first guess for survival chance, but we can make a more educated guess using the other features of the dataset.

So far, the models that we have seen are only able to deal with numerical variables. Here, only the variables **Age** and **Fare** are numerical. In order to use a perceptron or a logistic regression that takes into account the other variables, we would need to convert them to numerical variables. Usually a discrete variables takes one of k discrete values. Here are a few possible ways to encode them to numerical variables :
- **Numeric** : Assign each of the values to a number, say 1.0/k, 2.0/k,..., 1.0. This is a reasonable choice only when the discrete values do signify some sort of numeric quantities.
- **Thermometer code** : If your discrete values are ordered, from 1,...,k but not a natural mapping to real numbers, you can use a vector of length k binary variables, where we convert a discrete value $0<j\leq k$ into a vector in which the first j values are 1.0 and the rest are 0.0. This convey something about the ordering.
- **Factored code** : If your discrete value can be decomposed into two(ore more part), it can be best to treat those parts as separate features and choose an appropriate encoding for each of them.
- **One-hot encoding** : The best strategy when there is no information on the data is to use a vector of length k where we convert a discrete value $0<j\leq k$ into a vector for which all values are 0 except for the j-th which is 1.0

Q : If you are using a linear model, which of the Thermometer code and the One-hot encoding is more expressive ?

We restrict our attention to a limited_set of features :

In [ ]:
df = train.loc[:,['Pclass', 'Sex', 'Age', 'Fare','Embarked','Survived']]
df.head()

Sometimes, there are missing values in the dataset, this is not the topic of todays lecture, but here is an example of how to deal with them :

In [ ]:
df.isna().sum()

Here we can see that the value of the Age is missing for 177 passengers, we replace it by the mean value of the age of the passenger.(This is a solution, but not the only one). The value of Embarked is missing for two passengers, we select the most common value for them)

In [ ]:
df.Age.fillna(df.Age.mean(),inplace=True)
df.Embarked.fillna(df.Embarked.loc[df.Embarked.value_counts().argmax()], inplace=True)
df.isna().sum()

Now there are no more missing values and we can focus on dealing with the categorical variables
Here is a brief description of the variables of the dataframe :
- Pclass : The class of the ticket, it can be 1rd, 2nd or 3rd
- Sex : The sex of the Passenger
- Age : Age in Years
- Fare : Passenger Fare
- embarked : Port of Embarkation(C = Cherbourg, Q = Queenstown, S= Southampton)

TODO : For each of discrete variables, choose and implement an encoding of your choice
PS : make sure the output is of shape(num_samples, encoding_dimension)

In [ ]:
# Encoding of the Passenger class
# Pclass = df.Pclass.values

# Pclass_enc =

In [ ]:
#Encoding of the sex
# Sex = df.Sex.values

# Sex_enc =

In [ ]:
# Encoding of the Port of Embarkation
# embarked = df.embarked.values

#embarked_enc =

In [ ]:
# Obtain the numerical features and the target from the dataframe
X = df.loc[:,['Age', 'Fare']].values
y = df.loc[:,'Survived'].values

In [ ]:
# Add the encoded numerical variables
X_aug = np.hstack([X, Pclass_enc, Sex_enc, embarked_enc])

In order to evaluate the quality of our model, we could use it to make predictions on the test set, but we don't have access to the true values on this set (we could make a prediction on Kaggle to get a scoring). As before in this seminar, one way to get around this is to split the training data into training and validation data to evaluate the performance of the model on data it has never seen.

In [ ]:
id_train, id_val, y_train, y_val = train_test_split(list(range(len(X))),y,train_size=0.8,  random_state=44)
X_train = X[id_train]
X_val = X[id_val]
print(f' Training set size : {len(id_train)}\n Validation set size : {len(id_val)}')

In [ ]:
#X_train_aug = X_aug[id_train]
#X_val_aug = X_aug[id_val]

In [ ]:
# Accuracy of the Logistic Regression on the numerical variables
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
print(f'The accuracy of the logistic regression with the two numerical variables on the validation set is {(y_pred == y_val).mean()}')

In [ ]:
# Accuracy of the gender submission
y_pred = (df.Sex=='female').values[id_val]
print(f'The accuracy of the logistic regression with the two numerical variables on the validation set is {(y_pred == y_val).mean()}')

In [ ]:
# Accuracy of the augmented model
clf = LogisticRegression(max_iter=300)
clf.fit(X_train_aug, y_train)
y_pred = clf.predict(X_val_aug)
print(f'The accuracy of the logistic regression with the augmented model on the validation set is {(y_pred == y_val).mean()}')

Q : Should you do any preprocessing on the numerical variables ? Propose two different ideas.